In [1]:
import os
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

# Chain

In [2]:
from langchain_openai import ChatOpenAI

# model 
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 실행
llm.invoke("지구의 자전 주기는?")


AIMessage(content="지구의 자전 주기는 약 24시간입니다. 정확히는 23시간 56분 4초 정도로, 이를 '항성일'이라고 부릅니다. 우리가 일반적으로 사용하는 24시간은 태양이 지구를 기준으로 이동한 시간을 기준으로 한 '태양일'입니다. 태양일은 평균적으로 24시간으로 정의됩니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 15, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ba3090b-259e-4a3a-9bb2-161f60c242d7-0', usage_metadata={'input_tokens': 15, 'output_tokens': 81, 'total_tokens': 96, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 더 정확하게 말하자면, 평균적으로 23시간 56분 4초(약 86400초)입니다. 이 시간은 지구가 자전하여 한 번의 완전한 회전을 하는 데 걸리는 시간으로, 이를 기준으로 하루가 정의됩니다. 그러나 일상적으로는 24시간으로 표현됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 29, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bba3c8e70b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5a808fb4-162c-4934-8e70-e9cf2411d602-0', usage_metadata={'input_tokens': 29, 'output_tokens': 85, 'total_tokens': 114, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

"지구의 자전 주기는 약 24시간입니다. 정확히 말하면, 지구가 한 바퀴 자전하는 데 걸리는 시간은 약 23시간 56분 4초로, 이를 '항성일'이라고 합니다. 그러나 태양과의 상대적인 위치를 기준으로 할 때, 지구가 태양을 다시 같은 위치에 오도록 하기 위해 약간 더 긴 시간이 걸리며, 이를 '태양일'이라고 하여 약 24시간이 됩니다. 일반적으로 우리가 사용하는 하루는 이 태양일을 기준으로 합니다."

# Multi chain

In [6]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-4o-mini")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The Korean word "미래" translates to "future" in English.'

In [9]:
chain2 = (
    {"english_word": chain1}            # chain1 결과를 chain2의 input으로 사용 prompt template과 형식이 동일한듯
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'"미래"라는 한국어 단어는 영어로 "future"로 번역됩니다. 옥스퍼드 사전에서 "future"는 "the time that will come after the present" 즉, 현재 이후에 올 시간이라는 의미로 설명됩니다. 또한, "미래"는 어떤 사건이나 상황이 발생할 가능성이 있는 시점을 나타내며, 일반적으로 희망이나 계획, 예측과 관련된 개념으로 사용됩니다.'

# Runnable

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# 1. 컴포넌트 정의
prompt = ChatPromptTemplate.from_template("지구과학에서 {topic}에 대해 간단히 설명해주세요.")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

# 2. 체인 생성
chain = prompt | model | output_parser

# 3. invoke 메소드 사용
result = chain.invoke({"topic": "지구 자전"})
print("invoke 결과:", result)

/tmp/ipykernel_15030/1359593953.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


invoke 결과: 지구 자전은 지구가 자전축을 중심으로 회전하는 운동을 말합니다. 지구는 하루에 한 번, 약 24시간에 걸쳐 한 바퀴 회전합니다. 이 자전 운동은 지구의 북극과 남극을 연결하는 가상의 축을 중심으로 이루어지며, 지구의 자전 방향은 서쪽에서 동쪽으로 향합니다.

지구 자전의 결과로 낮과 밤이 생기며, 이는 태양의 빛이 지구의 한쪽 면에만 비추기 때문입니다. 자전 속도는 적도에서 가장 빠르고, 극지방에서는 속도가 거의 0에 가까워집니다. 또한, 지구 자전은 조수 간만의 차이와 같은 여러 자연 현상에도 영향을 미칩니다.

지구의 자전은 약 4.5억 년 전부터 계속되어 왔으며, 시간이 지남에 따라 자전 속도는 조금씩 변화하고 있습니다. 이러한 자전 운동은 지구의 기후와 생태계에도 중요한 역할을 합니다.


In [11]:
# batch 메소드 사용
topics = ["지구 공전", "화산 활동", "대륙 이동"]
results = chain.batch([{"topic": t} for t in topics])
for topic, result in zip(topics, results):
    print(f"{topic} 설명: {result[:50]}...")  # 결과의 처음 50자만 출력

지구 공전 설명: 지구 공전은 지구가 태양 주위를 한 바퀴 도는 운동을 의미합니다. 지구는 타원형 궤도를 따...
화산 활동 설명: 화산 활동은 지구 내부의 마그마가 지표면으로 분출되는 과정을 말합니다. 이 과정은 지구의 ...
대륙 이동 설명: 대륙 이동론은 지구의 대륙이 시간에 따라 이동해왔다는 이론입니다. 이 이론은 20세기 초 ...


In [12]:
# stream 메소드 사용
stream = chain.stream({"topic": "지진"})
print("stream 결과:")
for chunk in stream:
    print(chunk, end="", flush=True)
print()

stream 결과:
지진은 지구의 내부에서 발생하는 에너지 방출로 인해 발생하는 자연 현상입니다. 이 에너지는 지구의 판이 서로 충돌하거나 미끄러지면서 축적되고, 일정한 한계를 넘어서면 갑작스럽게 방출되어 지진파를 만들어냅니다. 

지진은 주로 지각의 판 경계에서 발생하는데, 이러한 경계는 크게 세 가지 유형으로 나뉩니다: 수렴 경계(판이 서로 충돌), 발산 경계(판이 멀어짐), 그리고 변환 경계(판이 서로 스치듯 움직임). 

지진의 강도는 리히터 규모로 측정되며, 이 규모는 지진이 발생할 때 방출된 에너지의 양을 나타냅니다. 지진은 지표면에 진동을 일으키고, 이로 인해 건물이나 구조물의 피해, 땅의 단층, 또는 쓰나미와 같은 2차 피해를 초래할 수 있습니다. 

지진의 예측은 매우 어려운 과학적 도전 중 하나이며, 지진 발생 후에는 피해를 최소화하기 위한 다양한 대응 방안이 필요합니다.


In [ ]:
import nest_asyncio
import asyncio

# nest_asyncio 적용 (구글 코랩 등 주피터 노트북에서 실행 필요)
nest_asyncio.apply()

# 비동기 메소드 사용 (async/await 구문 필요)
async def run_async():
    result = await chain.ainvoke({"topic": "해류"})
    print("ainvoke 결과:", result[:50], "...")

asyncio.run(run_async())

ainvoke 결과: 해류는 바다에서 물이 일정한 방향으로 흐르는 현상을 말합니다. 해류는 여러 가지 요인에 의 ...
